In [2]:
%load_ext autoreload
%autoreload 2
import numpy as np
from vvasp.VizClasses import NeuropixelsChronicHolder
from vvasp.atlas_utils import Atlas
import pyvista as pv
from labdata.schema import *

[2024-05-14 15:45:39,754][INFO]: Connecting mmelin@churchland-data.cmojfwfr0b9t.us-west-2.rds.amazonaws.com:3306
[2024-05-14 15:45:40,285][INFO]: Connected mmelin@churchland-data.cmojfwfr0b9t.us-west-2.rds.amazonaws.com:3306


In [3]:
SUBJECT = 'JC131'

ProbeInsertion() & f'subject_name LIKE "{SUBJECT}"'

subject_name unique mouse id,procedure_type Defines procedures that are not an experimental session,procedure_datetime,probe_id probe id to keep track or re-uses,insertion_ap anterior posterior distance from bregma,insertion_ml median lateral distance from bregma,insertion_depth insertion depth (how much shank is inserted from dura),insertion_el elevation of the probe (angle),insertion_az azimuth of the probe (angle),insertion_spin spin on the probe shanks
JC131,chronic implant,2023-07-19 20:00:00,20403312753,-3761.0,-2886.0,4500.0,120.0,-30.0,200.0


In [4]:
insertions2plot = Probe() * ProbeInsertion() & f'subject_name = "{SUBJECT}"'
#insertions2plot = Probe() * ProbeInsertion() & 'subject_name = "JC131"'
#insertions2plot = Probe() * ProbeInsertion() & 'subject_name = "JC164"'
insertions = insertions2plot.fetch(format='frame')
insertions.head()

,,,,probe_type,probe_n_shanks,probe_recording_channels,insertion_ap,insertion_ml,insertion_depth,insertion_el,insertion_az,insertion_spin
probe_id,subject_name,procedure_type,procedure_datetime,,,,,,,,,
20403312753,JC131,chronic implant,2023-07-19 20:00:00,24,4,384,-3761.0,-2886.0,4500.0,120.0,-30.0,200.0


In [23]:
plotter = pv.Plotter(notebook=True)
plotter.show()
#from pyvistaqt import BackgroundPlotter
#plotter = BackgroundPlotter()

atlas = Atlas(plotter, min_tree_depth=8, max_tree_depth=8)
#atlas.add_atlas_region_mesh('MOp')
atlas.add_atlas_region_mesh('LP')
#atlas.add_atlas_region_mesh('ACAd')
#atlas.add_atlas_region_mesh('ACAv')
atlas.add_atlas_region_mesh('VISp')
#atlas.add_atlas_region_mesh('CP')

probes = []
for i, entrypoint in insertions.iterrows():
    entry_point = entrypoint[['insertion_ml','insertion_ap']].values
    depth = entrypoint['insertion_depth']
    angles = entrypoint[['insertion_el','insertion_spin','insertion_az']].values
    if entrypoint.probe_type == '24':
        prb = NeuropixelsChronicHolder('NP24','head_fixed', plotter, root_intersection_mesh=atlas.meshes['root'])
    elif entrypoint.probe_type == '0':
        prb = NeuropixelsChronicHolder('NP1','head_fixed', plotter, root_intersection_mesh=atlas.meshes['root'])
    prb.drive_probe_from_entry(entry_point, angles, depth)
    prb.make_inactive()
    #prb.set_location(np.concatenate([entry_point,np.array([0])]), angles*0)
    probes.append(prb)

plotter.isometric_view()

Widget(value='<iframe src="http://localhost:49152/index.html?ui=P_0x18f83d9a1a0_3&reconnect=auto" class="pyvis…

In [24]:
from pathlib import Path
savedir = Path(r'C:\Data\churchland\chronic_manuscript_figs\raw_plots\trajectories')

#plotter.save_graphic(savedir / f'{SUBJECT}_trajectories.pdf', raster=False)
plotter.screenshot(savedir / f'{SUBJECT}_trajectories.png', return_img=False, scale=5)